Importing the required libraries


In [64]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import nltk
from nltk.stem.porter import PorterStemmer
import pickle

Loading the data files(csv files)

In [65]:
movies_data = pd.read_csv('/content/tmdb_5000_movies.csv')
credits_data=pd.read_csv('/content/tmdb_5000_credits.csv')

Top 5 rows of the movies_data

In [66]:
movies_data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


Top 5 rows of the credits_data

In [67]:
credits_data.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Observing the number of rows and columns in the data frames

---



In [68]:
movies_data.shape

(4803, 20)

In [69]:
credits_data.shape

(4803, 4)

Merging movies_data frame and credits_data frame on the basis of id

In [70]:
credits_data.columns=['id','title','cast','crew']
Movies = movies_data.merge(credits_data,on='id')

In [71]:
Movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Dropping the unnecessary columns from the data frame

In [72]:
#Changing the column name 'title_x' to 'title'
Movies['title']=Movies['title_x']
Movies = Movies.drop(columns=['original_title','title_x','title_y','homepage','budget','original_language','production_companies','production_countries','spoken_languages','status','release_date','revenue','runtime'])

In [73]:
Movies.head()

,genres,id,keywords,overview,popularity,tagline,vote_average,vote_count,cast,crew,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950,The Legend Ends,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another.",6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",John Carter


In [74]:
Movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        4803 non-null   object 
 1   id            4803 non-null   int64  
 2   keywords      4803 non-null   object 
 3   overview      4800 non-null   object 
 4   popularity    4803 non-null   float64
 5   tagline       3959 non-null   object 
 6   vote_average  4803 non-null   float64
 7   vote_count    4803 non-null   int64  
 8   cast          4803 non-null   object 
 9   crew          4803 non-null   object 
 10  title         4803 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 450.3+ KB


Finding popular movies


In [75]:
#Sorting the movies based on popularity in the descending order
popular_movies = Movies.sort_values('popularity',ascending=False)

In [76]:
popular_movies

,genres,id,keywords,overview,popularity,tagline,vote_average,vote_count,cast,crew,title
546,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 16, ""...",211672,"[{""id"": 3487, ""name"": ""assistant""}, {""id"": 179...","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses",6.4,4571,"[{""cast_id"": 22, ""character"": ""Scarlet Overkil...","[{""credit_id"": ""5431b2b10e0a2656e20026c7"", ""de...",Minions
95,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",157336,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...,8.1,10867,"[{""cast_id"": 9, ""character"": ""Joseph Cooper"", ...","[{""credit_id"": ""52fe4bbf9251416c910e4801"", ""de...",Interstellar
788,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",293660,"[{""id"": 2095, ""name"": ""anti hero""}, {""id"": 307...",Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending,7.4,10995,"[{""cast_id"": 99, ""character"": ""Wade Wilson / D...","[{""credit_id"": ""56c986b2925141172f0068b6"", ""de...",Deadpool
94,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",118340,"[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...","Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere.,7.9,9742,"[{""cast_id"": 1, ""character"": ""Peter Quill / St...","[{""credit_id"": ""538ce329c3a3687155003358"", ""de...",Guardians of the Galaxy
127,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",76341,"[{""id"": 2964, ""name"": ""future""}, {""id"": 3713, ...",An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day.,7.2,9427,"[{""cast_id"": 2, ""character"": ""Max Rockatansky""...","[{""credit_id"": ""577da370c3a36817f8003838"", ""de...",Mad Max: Fury Road
...,...,...,...,...,...,...,...,...,...,...,...
4625,"[{""id"": 27, ""name"": ""Horror""}]",426067,[],A Broadway producer puts on a play with a Devi...,0.001389,The hot spot where Satan's waitin'.,0.0,0,"[{""cast_id"": 1, ""character"": ""Dawn"", ""credit_i...","[{""credit_id"": ""582a4cdb9251417b1803dd47"", ""de...",Midnight Cabaret
4118,[],325140,[],"Raju, a waiter, is in love with the famous TV ...",0.001186,NaN,0.0,0,[],[],Hum To Mohabbat Karega
4727,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...",65448,"[{""id"": 378, ""name"": ""prison""}, {""id"": 209476,...",A hitchhiker named Martel Gordone gets in a fi...,0.001117,"There's only one way out, and 100 fools stand ...",4.9,8,"[{""cast_id"": 1000, ""character"": ""Sweet Pea"", ""...","[{""credit_id"": ""52fe470ac3a368484e0b1e1b"", ""de...",Penitentiary
3361,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 28, ""nam...",77156,[],A man who is having an affair with a married w...,0.000372,Don't you dare go in there!,4.0,3,"[{""cast_id"": 6, ""character"": ""Talmudge"", ""cred...","[{""credit_id"": ""52fe495fc3a368484e128181"", ""de...",Alien Zone


Most popular movies

In [77]:
popular_movies['title'].head(10)

546                                              Minions
95                                          Interstellar
788                                             Deadpool
94                               Guardians of the Galaxy
127                                   Mad Max: Fury Road
28                                        Jurassic World
199    Pirates of the Caribbean: The Curse of the Bla...
82                        Dawn of the Planet of the Apes
200                The Hunger Games: Mockingjay - Part 1
88                                            Big Hero 6
Name: title, dtype: object

Finding the top 10 movies of all time (Bayesian estimate)

In [78]:
mean_vote = Movies['vote_average'].mean()

In [79]:
min_votes= Movies['vote_count'].quantile(0.9)

In [80]:
top_movies = Movies.copy().loc[Movies['vote_count']>=min_votes]

In [81]:
def weighted_rating(x, m=min_votes, C=mean_vote):
    v = x['vote_count']
    R = x['vote_average']
    #Bayesian estimate
    return (v/(v+m) * R) + (m/(m+v) * C) 

In [82]:
top_movies['score']=top_movies.apply(weighted_rating,axis=1)

In [83]:
#Sorting the movies based on the score obtained
top_movies= top_movies.sort_values('score',ascending=False)

In [84]:
#Top 10 movies of all time from the data taken
top_movies['title'].head(10)

1881                         The Shawshank Redemption
662                                        Fight Club
65                                    The Dark Knight
3232                                     Pulp Fiction
96                                          Inception
3337                                    The Godfather
95                                       Interstellar
809                                      Forrest Gump
329     The Lord of the Rings: The Return of the King
1990                          The Empire Strikes Back
Name: title, dtype: object

Converting a dictionary to list

In [85]:
def convertToList(text):
    List = []
    for i in ast.literal_eval(text): #convert string object to list
        List.append(i['name']) 
    return List


In [86]:
#Converting genres which is in the form of dictionary to a list with genre names
Movies['genres'] = Movies['genres'].apply(convertToList)

In [87]:
Movies['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

Finding all the genres from the data 

In [88]:
genre_labels = set()
for genre in Movies['genres']:
    genre_labels = genre_labels.union(set(genre))

In [89]:
genre_labels

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

In [90]:
# Function that counts the number of times each of the genre keywords appear
def count_word(dataset, ref_col, census):
    keyword_count = dict()
    for s in census: 
        keyword_count[s] = 0
    for census_keywords in dataset[ref_col]:        
        if type(census_keywords) == float and pd.isnull(census_keywords): 
            continue        
        for s in [s for s in census_keywords if s in census]: 
            if pd.notnull(s): 
                keyword_count[s] += 1
    #______________________________________________________________________
    # convert the dictionary in a list to sort the keywords by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count

In [91]:
# Calling this function gives access to a list of genre keywords which are sorted by decreasing frequency
keyword_occurences, dum = count_word(Movies, 'genres', genre_labels)
keyword_occurences[:5]

[['Drama', 2297],
 ['Comedy', 1722],
 ['Thriller', 1274],
 ['Action', 1154],
 ['Romance', 894]]

In [92]:
#Converting genres to a string seperated with commas
Movies['Genres'] = [','.join(i) if isinstance(i, list) else i for i in Movies['genres']]

In [93]:
Movies['Genres'] 

0       Action,Adventure,Fantasy,Science Fiction
1                       Adventure,Fantasy,Action
2                         Action,Adventure,Crime
3                    Action,Crime,Drama,Thriller
4               Action,Adventure,Science Fiction
                          ...                   
4798                       Action,Crime,Thriller
4799                              Comedy,Romance
4800               Comedy,Drama,Romance,TV Movie
4801                                            
4802                                 Documentary
Name: Genres, Length: 4803, dtype: object

In [94]:
Movies['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [95]:
Movies['Genres'] = Movies['Genres'].fillna("").astype('str')

In [96]:
Movies['Genres']

0       Action,Adventure,Fantasy,Science Fiction
1                       Adventure,Fantasy,Action
2                         Action,Adventure,Crime
3                    Action,Crime,Drama,Thriller
4               Action,Adventure,Science Fiction
                          ...                   
4798                       Action,Crime,Thriller
4799                              Comedy,Romance
4800               Comedy,Drama,Romance,TV Movie
4801                                            
4802                                 Documentary
Name: Genres, Length: 4803, dtype: object

Removing all the stop_words which are in english like and,are, in ,on etc and convert it into a vector form

In [97]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(Movies['Genres'])
tfidf_matrix

<4803x276 sparse matrix of type '<class 'numpy.float64'>'
	with 20631 stored elements in Compressed Sparse Row format>

Tfid vector matrix

In [98]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.49309367, 0.29270708, 0.05927115],
       [0.49309367, 1.        , 0.17786505, 0.07470562],
       [0.29270708, 0.17786505, 1.        , 0.1918586 ],
       [0.05927115, 0.07470562, 0.1918586 , 1.        ]])

In [99]:
# Build a 1-dimensional array with movie titles
titles = Movies['title']
indices = pd.Series(Movies.index, index=Movies['title'])

In [100]:
# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(movie):
    idx = Movies[Movies['title'] == movie].index[0]
    # sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(list(enumerate(cosine_sim[idx])), key=lambda x: x[1], reverse=True)
    for i in sim_scores[1:6]:
      print(Movies.iloc[i[0]].title)

In [101]:
genre_recommendations('Titanic')

Fifty Shades of Grey
Angel Eyes
Fatal Attraction
Cruel Intentions
O


In [102]:
Movies.head()

,genres,id,keywords,overview,popularity,tagline,vote_average,vote_count,cast,crew,title,Genres
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Avatar,"Action,Adventure,Fantasy,Science Fiction"
1,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Pirates of the Caribbean: At World's End,"Adventure,Fantasy,Action"
2,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Spectre,"Action,Adventure,Crime"
3,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950,The Legend Ends,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",The Dark Knight Rises,"Action,Crime,Drama,Thriller"
4,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another.",6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",John Carter,"Action,Adventure,Science Fiction"


Fectching the director names from the crew and making into a list

In [103]:
def fetch_director(text):
    List = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            List.append(i['name'])
    return List

In [104]:
Movies['director'] = Movies['crew'].apply(fetch_director)

Taking the first 5 names from the dictionary and converting them into a list

In [105]:
def convert5ToList(text):
    List = []
    count = 0
    for i in ast.literal_eval(text):
        if count < 5:
            List.append(i['name'])
        count+=1
    return List 

In [106]:
#Converting the cast which is in the dictionary to list
Movies['cast'] = Movies['cast'].apply(convert5ToList)
Movies['cast'] = Movies['cast'].apply(lambda x:x[0:5])

In [107]:
#If the overview column is empty treat it as empty string
Movies['overview'] = Movies['overview'].fillna('')

In [108]:
Movies['overview'] = Movies['overview'].apply(lambda x:x.split())

In [109]:
#Converting keywords to list
Movies['keywords'] = Movies['keywords'].apply(convertToList)

Removing spaces between the strings to make into a single word

In [110]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [111]:
Movies['cast'] = Movies['cast'].apply(collapse)
Movies['director'] = Movies['director'].apply(collapse)
Movies['genres'] = Movies['genres'].apply(collapse)
Movies['keywords'] = Movies['keywords'].apply(collapse)

Combining all the columns to make a column named 'tags'

In [112]:
Movies['tags'] = Movies['overview'] + Movies['genres'] + Movies['keywords'] + Movies['cast'] + Movies['director']

In [113]:
Movies['tags'] = Movies['tags'].apply(lambda x: " ".join(x))

In [114]:
#Creating a PorterStemmer object
ps =PorterStemmer()

In [115]:
#Convert string to list and making all the similar words as a single word
#Ex: dancing,danced,dance all are considered as a similar single word
def Stem(text):
  L=[]
  for i in text.split():
    L.append(ps.stem(i))

  return " ".join(L)

In [116]:
Movies['tags']=Movies['tags'].apply(Stem)

In [117]:
cv = CountVectorizer(max_features=5000,stop_words='english')

onverting the tags to vectors

In [118]:
vector = cv.fit_transform(Movies['tags']).toarray()

Calculating the vector distances of every movie from every other movie

In [119]:
similarity_by_movie = cosine_similarity(vector)

In [120]:
similarity_by_movie.shape

(4803, 4803)

In [121]:
#Function for recommending movies(given the movie title)
def recommend(movie):
    index = Movies[Movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity_by_movie[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(Movies.iloc[i[0]].title)

In [122]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Independence Day
Falcon Rising
Battle: Los Angeles


In [123]:
recommend('Titanic')

The Notebook
Under the Same Moon
Ghost Ship
The Bounty
Pirates of the Caribbean: On Stranger Tides


Dumping the data frames to .pkl files

In [124]:
pickle.dump(Movies.to_dict(), open('movies_dict.pkl','wb'))

In [125]:
pickle.dump(similarity_by_movie,open('similarity_by_movie.pkl','wb'))

In [126]:
pickle.dump(cosine_sim,open('similarity_by_genre.pkl','wb'))